# Reaction via `qrules`

Address to issue ✅&nbsp;[ComPWA/gluex-nstar#5](https://github.com/ComPWA/gluex-nstar/issues/5), this section is an investigation of creating a virtual particle that represents the state $p \gamma$ in `QRules`

As an example of side note, create a virtual particle that represents state $e^+ e^-$ can be seen [here](https://qrules.readthedocs.io/0.10.x/usage/particle/#adding-custom-particle-definitions-through-python).

## Finding particles

Our reaction is $p + \gamma \to \eta + \pi^0 + p$, so we can look for each of the reaction and decay particles

In [ ]:
from __future__ import annotations

import graphviz
import qrules
from ampform.io import improve_latex_rendering
from IPython.display import Math
from qrules import InteractionType, StateTransitionManager, io
from qrules.particle import Particle, Spin, load_pdg

improve_latex_rendering()
particle_db = load_pdg()

The [`ParticleCollection`](https://qrules.readthedocs.io/0.10.x/api/qrules.particle/#qrules.particle.ParticleCollection) class offers some methods to search for particles by name or by PID, name, or other properties.

### Proton

Search by Particle ID (PID) for proton:

In [ ]:
particle_db.find(2212)

Search by Particle name for proton:

In [ ]:
particle_db.find("p")
proton = particle_db["p"]

In [ ]:
Math(proton.latex)

### $\gamma$

similarly for $\gamma$:

In [ ]:
particle_db.find("gamma")

In [ ]:
gamma = particle_db["gamma"]
Math(gamma.latex)

### $\eta$

In [ ]:
particle_db.find("eta")

In [ ]:
eta = particle_db["eta"]
Math(gamma.latex)

### $\pi^0$

In [ ]:
particle_db.find("pi0")

In [ ]:
pi0 = particle_db["pi0"]
Math(pi0.latex)

# Virtual particle that represents the state pγ 

## Adding custom particle

We add the virtual particle $p\gamma$ to represent the state of $p-\gamma$ system in the photoproduction reaction.
See PWA101 (temporary [here](https://compwa--217.org.readthedocs.build/report/999.html)), the value of the mass of the system $m_{p\gamma}$ in CM frame is about 4.102 GeV in this analysis.


We use `Particle` from `QRules` for adding custom particle, so that some quantum numbers we need to determine for the $p\gamma$ virtual state particle:

* (Total) Spin
    * $J_{p\gamma} = \frac{1}{2}$ or $\frac{3}{2}$
      
      because $J_{p\gamma}$ = from $|s_p-s_{\gamma}|$ to $s_p + s_{\gamma}$ = from $|\frac{1}{2}-1|$ to $\frac{1}{2} + 1$
      
* Mass
    * $m_{p\gamma} = E_{p\gamma} \approx 4.102$
    
* Charge
    * $Q_{p\gamma} = Q_p + Q_{\gamma} = 1 $
      
* Isospin and $3^{rd}$ component of isospin
    * $I_{p\gamma} = I_p = \frac{1}{2}$
    * $I_{3_{p\gamma}} = I_{3_p} = +\frac{1}{2}$
  
* Baryon Number
    * $B_{p\gamma} = B_p  = 1$

* Parity
    * $P_{p\gamma} = P_p \times P_{\gamma} = +1 \times (-1) = -1$

These quantum numbers provide a complete and accurate description of the virtual particle representing the p−γ system with a mass of 4.102 GeV.
The pγ virtual particle has two possibilities for the total spin, while the other quantum numbers are fixed.

We add the spin-$\frac{1}{2}$ $p\gamma$ virtual particle via `QRules` and named it $p\gamma$ with a make-up pid number 99990

In [ ]:
pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma

We add the spin-$\frac{3}{2}$ $p\gamma$ virtual particle via `QRules` and named it $p\gamma (s3/2)$ with a make-up pid number 99991

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)
pgamma2

In [ ]:
Math(pgamma.latex)

In [ ]:
Math(pgamma2.latex)

In [ ]:
particle_db.add(pgamma)

In [ ]:
particle_db.add(pgamma2)

The $`p\gamma`$ virtual states generated via `QRules` will be used in later stages, both possible spin states $(J= \frac{1}{2}$ and $\frac{3}{2})$ should be taken into account if possible, otherwise simplified if necessary.

:::{important}
It is possible to add any kind of custom Particle, as long as its quantum numbers comply with the [gellmann_nishijima()](https://qrules.readthedocs.io/0.10.x/api/qrules.conservation_rules/#qrules.conservation_rules.gellmann_nishijima) rule:
:::


:::{note}
[Gell-Mann-Nishijima formula](https://en.wikipedia.org/wiki/Gell-Mann–Nishijima_formula):
$Q = I_3 + \frac{1}{2}(B+S+C+B'+T)$

where 
Q is charge, 
$I_3$ is Spin projection of isospin, 
 B is baryon number, 
 S is strangeness, 
 C is charmness, 
 B' is bottomness, and 
 T is topness.
:::


In [ ]:
reaction1_all = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_interaction_types=["strong", "EM", "weak"],
    formalism="canonical-helicity",
    particle_db=particle_db,
)

In [ ]:
dot_all = qrules.io.asdot(reaction1_all, collapse_graphs=True)
graphviz.Source(dot_all)

In [ ]:
reaction1_strong_em = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
)

In [ ]:
dot_se = qrules.io.asdot(reaction1_strong_em, collapse_graphs=True)
graphviz.Source(dot_se)

In [ ]:
reaction1_strong = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_interaction_types=["strong"],
    formalism="canonical-helicity",
    particle_db=particle_db,
)

In [ ]:
dot_s = qrules.io.asdot(reaction1_strong, collapse_graphs=True)
graphviz.Source(dot_s)

In [ ]:
stm = StateTransitionManager(
    initial_state=["pgamma"],
    final_state=["eta", "pi0", "p"],
    formalism="canonical-helicity",
    particle_db=particle_db,
)

In [ ]:
problem_sets = stm.create_problem_sets()
sorted(problem_sets, reverse=True)

In [ ]:
some_problem_set = problem_sets[60.0][0]
dot = io.asdot(some_problem_set, render_node=True)
graphviz.Source(dot)

In [ ]:
some_problem_set = problem_sets[1][0]
dot = io.asdot(some_problem_set, render_node=True)
graphviz.Source(dot)

In [ ]:
some_problem_set = problem_sets[0.0001][1]
dot = io.asdot(some_problem_set, render_node=True)
graphviz.Source(dot)

In [ ]:
reaction = stm.find_solutions(problem_sets)

In [ ]:
print("found", len(reaction.transitions), "solutions!")
reaction.get_intermediate_particles().names

In [ ]:
stm.set_allowed_interaction_types([InteractionType.STRONG])
problem_sets = stm.create_problem_sets()
reaction = stm.find_solutions(problem_sets)

print("found", len(reaction.transitions), "solutions!")
reaction.get_intermediate_particles().names

In [ ]:
stm.set_allowed_interaction_types([InteractionType.STRONG, InteractionType.EM])
problem_sets = stm.create_problem_sets()
reaction = stm.find_solutions(problem_sets)

print("found", len(reaction.transitions), "solutions!")
reaction.get_intermediate_particles().names